Here is a summary of the notebook so it's easier for you to read: I wanted to experiment with transformers because it is the core of GPT and other LLMs, and it seems like it would be a very powerful tool for this kind of project. I used BERT, and I didn't have time to use a validation set, so I just have the final training accuracy - a whopping 91%! Maybe overfitting but I'm super happy with it considering it's my first time using transformers.

For some reason numpy on colab is too high version to be compatible with BERT transformer, so I had to manually install a lower version.

In [ ]:
!pip install numpy==1.26.4
import os
os._exit(00)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip install emoji
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 10.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import torch as nn
import emoji
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from transformers import BertTokenizer
from datasets import Dataset

The data I downloaded also came from an excel file and was for some reason already labeled, with the emojis replaced with numbers. 0 - heart, 1 - baseball, 2 - smile, 3 - disappointed, 4 - fork and knife

In [ ]:
url = 'emojify_data.csv'
df = pd.read_csv(url)

valid = ['French macaroon is so tasty', '4']
for col in df.columns:
    if col not in valid:
        df.drop(col, axis=1, inplace=True)

df.head(10)

,French macaroon is so tasty,4
0,work is horrible,3
1,I am upset,3
2,throw the ball,1
3,Good joke,2
4,what is your favorite baseball game,1
5,I cooked meat,4
6,stop messing around,3
7,I want chinese food,4
8,Let us go play baseball,1
9,you are failing this exercise,3


In [ ]:
df_dataset = Dataset.from_pandas(df)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df_dataset = df_dataset.rename_column('French macaroon is so tasty', 'text')
df_dataset = df_dataset.rename_column('4', 'label')

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

tokenized_dataset = df_dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/182 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(tokenized_dataset, batch_size=4, shuffle=True)

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cpu


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)



model.train()
for epoch in range(3):
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total
    print(f"Epoch {epoch + 1} - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.4f}")

Epoch 1 - Loss: 1.5662 - Accuracy: 0.2802
Epoch 2 - Loss: 1.1078 - Accuracy: 0.6648
Epoch 3 - Loss: 0.5207 - Accuracy: 0.9121


Future improvements would definitely be increasing the number of emojis so Omar can show more than 4 reactions! would be really simple to code though, just increase number of classes and import that data so I'm optimistic about this model. Thank you for your time in reading.